In [ ]:
from __future__ import (absolute_import, division, print_function)
import os
import json

import matplotlib as mpl
import matplotlib.pyplot as plt

from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
from geopandas import GeoSeries, GeoDataFrame

%matplotlib inline

from sqlalchemy import create_engine
import psycopg2
import psycopg2.extras

In [ ]:
def pgconnect_wk09(credential_filepath):
    try:
        with open(credential_filepath) as f:
            db_conn_dict = json.load(f)
        conn = psycopg2.connect(**db_conn_dict)
        print('connected')
    except Exception as e:
        print("unable to connect to the database")
        print(e)
        return None
    return conn

def pgquery_wk09( conn, sqlcmd, args=None, msg=False, returntype='tuple'):
    """ utility function to execute some SQL query statement
        it can take optional arguments (as a dictionary) to fill in for placeholders in the SQL
        will return the complete query result as return value - or in case of error: None
        error and transaction handling built-in (by using the 'with' clauses)"""
    retval = None
    with conn:
        cursortype = None if returntype != 'dict' else psycopg2.extras.RealDictCursor
        with conn.cursor(cursor_factory=cursortype) as cur:
            try:
                if args is None:
                    cur.execute(sqlcmd)
                else:
                    cur.execute(sqlcmd, args)
                if (cur.description != None ):
                    retval = cur.fetchall() # we use fetchall() as we expect only _small_ query results
                if msg != False:
                    print("success: " + msg)
            except psycopg2.DatabaseError as e:
                if e.pgcode != None:
                    if msg: print("db read error: "+msg)
                    print(e)
            except Exception as e:
                print(e)
    return retval

In [ ]:
# edit this file and insert your unikey and sid
credfilepath = "data2x01_db.json"
conn_wk09 = pgconnect_wk09(credfilepath)

In [ ]:
postgis_check = '''
SELECT PostGIS_Version();
'''
pgquery_wk09(conn_wk09,postgis_check)

In [ ]:
schema_creation = '''CREATE SCHEMA IF NOT EXISTS assignment;'''
pgquery_wk09(conn_wk09, "DROP SCHEMA IF EXISTS assignment CASCADE", msg="cleared old schema")
pgquery_wk09(conn_wk09, schema_creation, msg="created schema assignment")

#schema_permission1 = '''GRANT ALL ON SCHEMA assignment TO y20s1d2x01_mile3211;'''
#schema_permission2 = '''GRANT ALL ON SCHEMA assignment TO y20s1d2x01_ktru7823;'''
#schema_permission3 = '''GRANT ALL ON SCHEMA assignment TO y20s1d2x01_xidu5391;'''
#pgquery_wk09(conn_wk09, schema_permission1, msg="permission granted")
#pgquery_wk09(conn_wk09, schema_permission2, msg="permission granted")
#pgquery_wk09(conn_wk09, schema_permission3, msg="permission granted")

In [ ]:
health_services_df = pd.read_csv('HealthServices.csv')
neighbourhoods_df = pd.read_csv('Neighbourhoods.csv')
nsw_postcodes_df = pd.read_csv('NSW_Postcodes.csv')
population_stats_df = pd.read_csv('PopulationStats2016.csv')
statistical_areas_df = pd.read_csv('StatisticalAreas.csv')

In [ ]:
statistical_areas_schema = '''CREATE TABLE IF NOT EXISTS assignment.statistical_areas (
    area_id           VARCHAR(20) PRIMARY KEY,
    area_name         VARCHAR(100),
    parent_area_id    VARCHAR(20)
)'''

table_name = "assignment.statistical_areas"
pgquery_wk09(conn_wk09, "DROP TABLE IF EXISTS "+table_name, msg="cleared old table")
pgquery_wk09(conn_wk09, statistical_areas_schema, msg="created "+table_name)

insert_stmt = """INSERT INTO assignment.statistical_areas VALUES ( %(area_id)s, %(area_name)s, %(parent_area_id)s
                    )"""

for index, row in statistical_areas_df.iterrows():
    pgquery_wk09(conn_wk09, insert_stmt, args=row)

# expect 414 rows
result = pgquery_wk09(conn_wk09, 'SELECT COUNT(*) FROM '+table_name)
result

In [ ]:
# use url for covid data?
import requests
url = 'https://data.nsw.gov.au/data/dataset/5424aa3b-550d-4637-ae50-7f458ce327f4/resource/227f6b65-025c-482c-9f22-a25cf1b8594f/download/covid-19-tests-by-date-and-location-and-result.csv'  

req = requests.get(url)
covid_csv_data = open('covid-19-tests-by-date-and-location-and-result.csv', 'wb').write(req.content)

covid_tests_df = pd.read_csv('covid-19-tests-by-date-and-location-and-result.csv')

# or just use downloaded data
# covid_tests_df = pd.read_csv('covid-19-tests-by-date-and-location-and-result-may-8.csv')

# covid_tests_clean_df = covid_tests_df.dropna();
covid_tests_clean_df = covid_tests_df.dropna(subset=['postcode'], inplace=False)

# convert postcode to ints
# semi convoluted manner, but directly changing it throws a warning for some reason
covid_tests_clean_df2 = covid_tests_clean_df.copy(deep=True)
covid_tests_clean_df2["postcode"] = covid_tests_clean_df["postcode"].astype(int)
covid_tests_clean_df = covid_tests_clean_df2
covid_tests_clean_df.shape

In [ ]:
covid_tests_schema = '''CREATE TABLE IF NOT EXISTS assignment.covid_19_tests (
    test_date       DATE NOT NULL,
    postcode        VARCHAR(4) NOT NULL,
    lhd_2010_code   CHAR(4),
    lhd_2010_name   VARCHAR(100),
    lga_code19      FLOAT,
    lga_name19      VARCHAR(100),
    result          VARCHAR(20) NOT NULL
)'''

table_name = "assignment.covid_19_tests"
pgquery_wk09(conn_wk09, "DROP TABLE IF EXISTS "+table_name, msg="cleared old table")
pgquery_wk09(conn_wk09, covid_tests_schema, msg="created "+table_name)

insert_stmt = """INSERT INTO assignment.covid_19_tests (test_date, postcode, lhd_2010_code, lhd_2010_name,
                    lga_code19, lga_name19, result)
                    SELECT %(test_date)s, %(postcode)s, %(lhd_2010_code)s,
                    %(lhd_2010_name)s, %(lga_code19)s, %(lga_name19)s, %(result)s;"""

# this takes forever but pls do not interrupt
for index, row in covid_tests_clean_df.iterrows():
    pgquery_wk09(conn_wk09, insert_stmt, args=row)

# TODO: is there a faster way
# expect 200 000+ rows (may 8)
# expect 400 000+ rows (recent)
result = pgquery_wk09(conn_wk09, 'SELECT COUNT(*) FROM '+table_name)
result

# fixing up the table

# a possible foreign key? but postcodes aren't unique
# ALTER TABLE assignment.covid_19_tests
#    ADD CONSTRAINT FK_covid_tests_postcode FOREIGN KEY (postcode) REFERENCES assignment.nsw_postcodes(postcode)

alter_table_command = '''ALTER TABLE assignment.covid_19_tests
                            ALTER COLUMN lga_code19 TYPE CHAR(5) USING CAST(ROUND(lga_code19) AS CHAR(5));'''
pgquery_wk09(conn_wk09, alter_table_command)

index_command = "CREATE INDEX covid_tests_idx ON assignment.covid_19_tests(postcode, result)"
result = pgquery_wk09(conn_wk09, index_command, returntype='dict')

In [ ]:
nsw_shp_schema = '''CREATE TABLE assignment.nsw_shp (
                        sa2_main16   VARCHAR(20) PRIMARY KEY,
                        sa2_5dig16   VARCHAR(20),
                        sa2_name16   VARCHAR(50),
                        sa3_code16   VARCHAR(20),
                        sa3_name16   VARCHAR(50),
                        sa4_code16   VARCHAR(20),
                        sa4_name16   VARCHAR(50),
                        gcc_code16   CHAR(5),
                        gcc_name16   VARCHAR(50),
                        ste_code16   VARCHAR(20),
                        ste_name16   VARCHAR(50),
                        area_sq_km16 FLOAT,
                        geometry     GEOMETRY(MULTIPOLYGON, 4283)
)''' 

table_name = "assignment.nsw_shp"
pgquery_wk09(conn_wk09, "DROP TABLE IF EXISTS "+table_name+" CASCADE", msg="cleared old table")
pgquery_wk09(conn_wk09, nsw_shp_schema, msg="created "+table_name)

nsw_shp_df = gpd.read_file( "1270055001_sa2_2016_aust_shape/SA2_2016_AUST.shp" )
nsw_shp_df['geom_wkt'] = nsw_shp_df['geometry'].apply(lambda x: x.wkt if x is not None else x)

insert_stmt = """INSERT INTO assignment.nsw_shp VALUES ( %(SA2_MAIN16)s, %(SA2_5DIG16)s, %(SA2_NAME16)s,
                    %(SA3_CODE16)s, %(SA3_NAME16)s, %(SA4_CODE16)s, %(SA4_NAME16)s, %(GCC_CODE16)s,
                    %(GCC_NAME16)s, %(STE_CODE16)s, %(STE_NAME16)s, %(AREASQKM16)s,
                    ST_Multi(ST_GeomFromText( %(geom_wkt)s,4283)) )"""

for idx, location in nsw_shp_df.iterrows():
    pgquery_wk09(conn_wk09, insert_stmt, args=location)



index_command = "CREATE INDEX nsw_shp_idx ON assignment.nsw_shp USING GIST (geometry)"
pgquery_wk09(conn_wk09, index_command, returntype='dict')

In [ ]:
# expect 2310 rows
result = pgquery_wk09(conn_wk09, 'SELECT COUNT(*) FROM '+table_name)
result

In [ ]:
commute_filepath = "Commutes2011.json"
commutes_df = pd.read_json(commute_filepath)
commutes_df.head()

In [ ]:
from pandas import json_normalize

commutes_expanded_df = pd.DataFrame(columns = ["origin", "destination", "people"])

# this takes a couple of minutes but pls do not interrupt
for index, row in commutes_df.iterrows():
    expanded_row = pd.concat([pd.DataFrame(json_normalize(x)) for x in row["destinations"]],ignore_index=True)
    origin_value = row["origin"]
    expanded_row["origin"] = origin_value
    expanded_row = expanded_row[["origin", "destination", "people"]]
    commutes_expanded_df = commutes_expanded_df.append(expanded_row)

commutes_grouped_df = commutes_expanded_df.groupby(["origin","destination"]).sum().reset_index()
commutes_grouped_df.shape

In [ ]:
commutes_schema = '''CREATE TABLE IF NOT EXISTS assignment.commutes (
    origin       VARCHAR(20),
    destination  VARCHAR(20),
    people       INTEGER,
    CONSTRAINT PK_commutes PRIMARY KEY (origin, destination)
)'''

table_name = "assignment.commutes"
pgquery_wk09(conn_wk09, "DROP TABLE IF EXISTS "+table_name, msg="cleared old table")
pgquery_wk09(conn_wk09, commutes_schema, msg="created "+table_name)

insert_stmt = """INSERT INTO assignment.commutes VALUES ( %(origin)s, %(destination)s, %(people)s
                )"""

# this takes a while but pls do not interrupt
for idx, row in commutes_grouped_df.iterrows():
    origin = str(row["origin"])
    destination = str(row["destination"])
    people = str(row["people"])
    pgquery_wk09(conn_wk09,
                 """INSERT INTO assignment.commutes VALUES ( """ + origin + """,
                         """ + destination + """, """ + people + """)""",
                 args=row)

# expect 65065 rows
result = pgquery_wk09(conn_wk09, 'SELECT COUNT(*) FROM '+table_name)
result

In [ ]:
health_services_schema = '''CREATE TABLE IF NOT EXISTS assignment.health_services (
    id          INTEGER PRIMARY KEY,
    name        VARCHAR(100),
    category    VARCHAR(100),
    num_beds    FLOAT,
    address     VARCHAR(1000),
    suburb      VARCHAR(100),
    state       CHAR(3),
    postcode    CHAR(4) NOT NULL,
    latitude    FLOAT   NOT NULL,
    longitude   FLOAT   NOT NULL,
    comment     VARCHAR(4000),
    website     VARCHAR(200)
)'''

table_name = "assignment.health_services"
pgquery_wk09(conn_wk09, "DROP TABLE IF EXISTS "+table_name, msg="cleared old table")
pgquery_wk09(conn_wk09, health_services_schema, msg="created "+table_name)

insert_stmt = """INSERT INTO assignment.health_services VALUES ( %(id)s, %(name)s, %(category)s,
                    %(num_beds)s, %(address)s, %(suburb)s, %(state)s, %(postcode)s,
                    %(latitude)s, %(longitude)s, %(comment)s, %(website)s
                    )"""

for index, row in health_services_df.iterrows():
    pgquery_wk09(conn_wk09, insert_stmt, args=row)

# expect 3026 rows
result = pgquery_wk09(conn_wk09, 'SELECT COUNT(*) FROM '+table_name)
result

health_services_add_column = '''ALTER TABLE assignment.health_services
                               ADD point GEOMETRY(POINT, 4283)
'''
pgquery_wk09(conn_wk09, health_services_add_column, msg="added column in "+table_name)

health_services_update_values = '''UPDATE assignment.health_services
                                SET point = ST_SetSRID(ST_MakePoint(longitude, latitude), 4283)
'''
pgquery_wk09(conn_wk09, health_services_update_values, msg="updated values in "+table_name)

# spatial index for health services
index_command = "CREATE INDEX health_services_pt_idx ON assignment.health_services USING GIST (point)"
result = pgquery_wk09(conn_wk09, index_command, returntype='dict')
result

In [ ]:
# pandas uses NaN (a float) to represent missing values, so the data types
# (population, dwellings, businesses, median_income, avg_monthly_rent) need to be floats
neighbourhoods_schema = '''CREATE TABLE IF NOT EXISTS assignment.neighbourhoods (
    area_id           VARCHAR(20) PRIMARY KEY REFERENCES assignment.nsw_shp(sa2_main16),
    area_name         VARCHAR(100),
    land_area         FLOAT,
    population        FLOAT,
    dwellings         FLOAT,
    businesses        FLOAT,
    median_income     FLOAT,
    avg_monthly_rent  FLOAT
)'''

table_name = "assignment.neighbourhoods"
pgquery_wk09(conn_wk09, "DROP TABLE IF EXISTS "+table_name+" CASCADE", msg="cleared old table")
pgquery_wk09(conn_wk09, neighbourhoods_schema, msg="created "+table_name)

insert_stmt = """INSERT INTO assignment.neighbourhoods VALUES ( %(area_id)s, %(area_name)s, %(land_area)s,
                    %(population)s, %(number_of_dwellings)s, %(number_of_businesses)s,
                    %(median_annual_household_income)s, %(avg_monthly_rent)s
                    )"""

for index, row in neighbourhoods_df.iterrows():
    pgquery_wk09(conn_wk09, insert_stmt, args=row)

# cleaning
update_command = """UPDATE assignment.neighbourhoods
SET population = 0
WHERE population = 'NaN';"""
pgquery_wk09(conn_wk09, update_command)

# expect 312 rows
result = pgquery_wk09(conn_wk09, 'SELECT COUNT(*) FROM '+table_name)
result

In [ ]:
nsw_postcodes_schema = '''CREATE TABLE IF NOT EXISTS assignment.nsw_postcodes (
    id        INTEGER PRIMARY KEY,
    postcode  CHAR(4) NOT NULL,
    locality  VARCHAR(50),
    longitude FLOAT,
    latitude  FLOAT
)'''

table_name = "assignment.nsw_postcodes"
pgquery_wk09(conn_wk09, "DROP TABLE IF EXISTS "+table_name, msg="cleared old table")
pgquery_wk09(conn_wk09, nsw_postcodes_schema, msg="created "+table_name)

insert_stmt = """INSERT INTO assignment.nsw_postcodes VALUES ( %(id)s, %(postcode)s, %(locality)s,
                    %(longitude)s, %(latitude)s
                    )"""

for index, row in nsw_postcodes_df.iterrows():
    pgquery_wk09(conn_wk09, insert_stmt, args=row)

index_command = "CREATE INDEX nsw_postcodes_idx ON assignment.nsw_postcodes(postcode)"
pgquery_wk09(conn_wk09, index_command, returntype='dict')

# expect 5639 rows
result = pgquery_wk09(conn_wk09, 'SELECT COUNT(*) FROM '+table_name)
result

In [ ]:
population_stats_schema = '''CREATE TABLE IF NOT EXISTS assignment.population_stats_2016 (
    area_id         VARCHAR(20) PRIMARY KEY REFERENCES assignment.nsw_shp(sa2_main16),
    area_name       VARCHAR(100),
    "0-4"           INTEGER,
    "5-9"           INTEGER,
    "10-14"         INTEGER,
    "15-19"         INTEGER,
    "20-24"         INTEGER,
    "25-29"         INTEGER,
    "30-34"         INTEGER,
    "35-39"         INTEGER,
    "40-44"         INTEGER,
    "45-49"         INTEGER,
    "50-54"         INTEGER,
    "55-59"         INTEGER,
    "60-64"         INTEGER,
    "65-69"         INTEGER,
    "70-74"         INTEGER,
    "75-79"         INTEGER,
    "80-84"         INTEGER,
    "85_and_over"   INTEGER,
    total_persons   INTEGER,
    females         INTEGER,
    males           INTEGER
)'''

table_name = "assignment.population_stats_2016"
pgquery_wk09(conn_wk09, "DROP TABLE IF EXISTS "+table_name, msg="cleared old table")
pgquery_wk09(conn_wk09, population_stats_schema, msg="created "+table_name)

insert_stmt = """INSERT INTO assignment.population_stats_2016 VALUES ( %(area_id)s, %(area_name)s,
                    %(0-4)s, %(5-9)s, %(10-14)s, %(15-19)s, %(20-24)s, %(25-29)s, %(30-34)s, %(35-39)s,
                    %(40-44)s, %(45-49)s, %(50-54)s, %(55-59)s, %(60-64)s, %(65-69)s, %(70-74)s, %(75-79)s,
                    %(80-84)s, %(85_and_over)s, %(total_persons)s, %(females)s, %(males)s
                    )"""

for index, row in population_stats_df.iterrows():
    pgquery_wk09(conn_wk09, insert_stmt, args=row)

# expect 576 rows
result = pgquery_wk09(conn_wk09, 'SELECT COUNT(*) FROM '+table_name)
result

In [ ]:
# population density = population divided by neighbourhood's land area
# population age = proportion of people aged 70+ (set to zero if no population)
# health service density = number of health services per neighbourhood per 1000 people
# hospital bed density = number of hospital beds per neighbourhood per 1000 people
# commuter density = number of people commuting per neighbourhood per (1000 people?)

all_query = """
WITH
neighbourhoods_with_geom AS (
    SELECT area_id, population, geometry
    FROM (assignment.neighbourhoods JOIN assignment.nsw_shp ON (area_id = sa2_main16))
),
pop_dens AS (
    SELECT area_id, (population / land_area) AS population_density
    FROM assignment.neighbourhoods
),
pop_age AS (
    SELECT area_id, 
            COALESCE(( CAST("70-74" AS FLOAT)
                        + CAST("75-79" AS FLOAT)
                        + CAST("80-84" AS FLOAT)
                        + CAST("85_and_over" AS FLOAT) )
                        / NULLIF(CAST("total_persons" AS FLOAT), 0), 0)
                        AS population_age
    FROM assignment.population_stats_2016 A
    WHERE EXISTS (SELECT 1
                    FROM assignment.neighbourhoods B
                    WHERE A.area_id = B.area_id)
),
health_dens AS (
    SELECT area_id,
        CASE
            WHEN population = 0
            THEN 0
            ELSE CAST(COUNT(area_id) AS FLOAT) / (population/1000.0)
        END AS health_service_density,
        CASE
            WHEN population = 0
            THEN 0
            ELSE SUM(COALESCE(NULLIF(num_beds, 'NaN'), 0)) / (population/1000.0)
        END AS hospital_bed_density
    FROM neighbourhoods_with_geom A LEFT JOIN assignment.health_services B
            ON (ST_Intersects(A.geometry, B.point))
    GROUP BY area_id, population
),
comm_dens AS (
    SELECT area_id,
            CAST(total_commuters AS FLOAT) / (population/1000.0)
                AS commuter_density
    FROM assignment.neighbourhoods LEFT JOIN (SELECT origin, SUM(people) AS total_commuters
                                FROM assignment.commutes
                                GROUP BY origin
                                ORDER BY origin) AS commutes_by_origin
            ON (origin = area_id)
    GROUP BY area_id, total_commuters
)

SELECT *
INTO assignment.vulnerability_measures
FROM pop_dens NATURAL JOIN pop_age NATURAL JOIN health_dens NATURAL JOIN comm_dens
ORDER BY area_id;
"""

table_name = "assignment.vulnerability_measures"
pgquery_wk09(conn_wk09, "DROP TABLE IF EXISTS "+table_name, msg="cleared old table")
pgquery_wk09(conn_wk09, all_query, msg="created "+table_name)

pk_command = '''ALTER TABLE assignment.vulnerability_measures ADD PRIMARY KEY (area_id);'''
pgquery_wk09(conn_wk09, pk_command)

fk_command = '''ALTER TABLE assignment.vulnerability_measures
                ADD CONSTRAINT fk_vuln_measures FOREIGN KEY (area_id) REFERENCES assignment.neighbourhoods(area_id);'''
pgquery_wk09(conn_wk09, fk_command)

# expect 312 rows, one for each area_id in neighbourhoods in sydney
result = pgquery_wk09(conn_wk09, 'SELECT COUNT(*) FROM '+table_name)
result

In [ ]:
# vulnerability = S( z(population density) + z(population age) - z(healthservice density) - z(hospitalbed density) )
# z(measure; x) = ( x - avg(measure) ) / std.dev(measure)

z_score_query = """WITH
area_ids AS (
    SELECT ROW_NUMBER() OVER () AS "id", area_id
        FROM assignment.vulnerability_measures
),
pd_z AS (
    SELECT ROW_NUMBER() OVER () AS "id",
                (population_density - (AVG(population_density) OVER ()) )
                / (STDDEV_POP(population_density) OVER ()) AS pd_z_score
        FROM assignment.vulnerability_measures
),
pa_z AS (
    SELECT ROW_NUMBER() OVER () AS "id",
                (population_age - (AVG(population_age) OVER ()) )
                / (STDDEV_POP(population_age) OVER ()) AS pa_z_score
        FROM assignment.vulnerability_measures
),
hsd_z AS (
    SELECT ROW_NUMBER() OVER () AS "id",
                    (health_service_density - (AVG(health_service_density) OVER ()) )
                    / (STDDEV_POP(health_service_density) OVER ()) AS hsd_z_score
        FROM assignment.vulnerability_measures
),
hbd_z AS (
    SELECT ROW_NUMBER() OVER () AS "id",
                    (hospital_bed_density - (AVG(hospital_bed_density) OVER ()) )
                    / (STDDEV_POP(hospital_bed_density) OVER ()) AS hbd_z_score
        FROM assignment.vulnerability_measures
),
cd_z AS (
    SELECT ROW_NUMBER() OVER () AS "id",
                    COALESCE((commuter_density - (AVG(commuter_density) OVER ()) )
                    / (STDDEV_POP(commuter_density) OVER ()), 0) AS cd_z_score
        FROM assignment.vulnerability_measures
)

SELECT area_id, pd_z_score, pa_z_score, hsd_z_score, hbd_z_score, cd_z_score
INTO assignment.vulnerability_z_scores
FROM area_ids NATURAL JOIN pd_z NATURAL JOIN pa_z
        NATURAL JOIN hsd_z NATURAL JOIN hbd_z NATURAL JOIN cd_z;"""

table_name = "assignment.vulnerability_z_scores"
pgquery_wk09(conn_wk09, "DROP TABLE IF EXISTS "+table_name, msg="cleared old table")
pgquery_wk09(conn_wk09, z_score_query, msg="created "+table_name)

pk_command = '''ALTER TABLE assignment.vulnerability_z_scores ADD PRIMARY KEY (area_id);'''
pgquery_wk09(conn_wk09, pk_command)

fk_command = '''ALTER TABLE assignment.vulnerability_z_scores
                ADD CONSTRAINT fk_z_score FOREIGN KEY (area_id) REFERENCES assignment.neighbourhoods(area_id);'''
pgquery_wk09(conn_wk09, fk_command)

# expect 312 rows
result = pgquery_wk09(conn_wk09, 'SELECT COUNT(*) FROM '+table_name)
result

In [ ]:
vulnerability_final_score_schema = '''
SELECT area_id,
        (1 / (1 + exp(-(pd_z_score + pa_z_score - hsd_z_score - hbd_z_score)))) AS vuln_score_no_commute,
        (1 / (1 + exp(-(pd_z_score + pa_z_score - hsd_z_score - hbd_z_score + cd_z_score)))) AS final_vuln_score
INTO assignment.vulnerability_final_scores
FROM assignment.vulnerability_z_scores;
'''

table_name = "assignment.vulnerability_final_scores"
pgquery_wk09(conn_wk09, "DROP TABLE IF EXISTS "+table_name, msg="cleared old table")
pgquery_wk09(conn_wk09, vulnerability_final_score_schema, msg="created "+table_name)

pk_command = '''ALTER TABLE assignment.vulnerability_final_scores ADD PRIMARY KEY (area_id);'''
pgquery_wk09(conn_wk09, pk_command)

fk_command = '''ALTER TABLE assignment.vulnerability_final_scores
                ADD CONSTRAINT fk_vulnerability_final_scores FOREIGN KEY (area_id) REFERENCES assignment.neighbourhoods(area_id);'''
pgquery_wk09(conn_wk09, fk_command)

# expect 312 rows
result = pgquery_wk09(conn_wk09, 'SELECT COUNT(*) FROM '+table_name)
result

In [ ]:
correlation_query = """
WITH
neighbourhoods_with_geom AS (
    SELECT area_id, gcc_code16, population, geometry
         FROM (assignment.neighbourhoods JOIN assignment.nsw_shp ON (area_id = sa2_main16))
),
postcode_with_geom AS (
SELECT postcode, AVG(longitude) AS avg_longitude, AVG(latitude) AS avg_latitude,
                ST_SetSRID(ST_MakePoint(AVG(longitude), AVG(latitude)), 4283) AS point
            FROM assignment.nsw_postcodes
            GROUP BY postcode
            ORDER BY postcode
),
cases_by_postcode_with_geom AS (
    SELECT postcode, positive_result_count, avg_longitude, avg_latitude, point
    FROM (SELECT postcode, result,
                COUNT(result) AS positive_result_count
            FROM assignment.covid_19_tests
            WHERE result = 'Case - Confirmed'
            GROUP BY postcode, result
            ORDER BY postcode) AS positive_cases
        JOIN postcode_with_geom USING (postcode)
),
tests_by_postcode_with_geom AS (
    SELECT postcode, test_count, avg_longitude, avg_latitude, point
    FROM (SELECT postcode, result,
                COUNT(result) AS test_count
            FROM assignment.covid_19_tests
            GROUP BY postcode, result
            ORDER BY postcode) AS total_tests_by_postcode
        JOIN postcode_with_geom USING (postcode)
),
cases_by_area_id AS (
    SELECT area_id, SUM(positive_result_count) AS positive_result_count
    FROM neighbourhoods_with_geom A LEFT JOIN cases_by_postcode_with_geom B
                    ON (ST_Intersects(A.geometry, B.point))
    GROUP BY area_id
    ORDER BY area_id
),
tests_by_area_id AS (
    SELECT area_id, SUM(test_count) AS total_test_count
    FROM neighbourhoods_with_geom A LEFT JOIN tests_by_postcode_with_geom B
                    ON (ST_Intersects(A.geometry, B.point))
    GROUP BY area_id
    ORDER BY area_id
)

SELECT positive_result_count AS "Positive COVID-19 cases", total_test_count AS "Total COVID-19 tests",
            vuln_score_no_commute AS "Vulnerability Score (no commutes)", final_vuln_score AS "Vulnerability Score"
FROM (cases_by_area_id
        JOIN tests_by_area_id USING (area_id))
        JOIN assignment.vulnerability_final_scores USING (area_id);
"""

correlation_df = pd.read_sql(correlation_query, conn_wk09)
correlation_df.corr()

In [ ]:
correlation_df.plot.scatter(x='Positive COVID-19 cases',y='Vulnerability Score')
plt.savefig('figure_scatter1.png')

In [ ]:
# well a lot of the commute counts were null so its pretty similar
correlation_df.plot.scatter(x='Positive COVID-19 cases',y='Vulnerability Score (no commutes)')
plt.savefig('figure_scatter2.png')

In [ ]:
correlation_df.plot.scatter(x='Total COVID-19 tests',y='Vulnerability Score')

In [ ]:
correlation_df.plot.scatter(x='Total COVID-19 tests',y='Vulnerability Score (no commutes)')

In [ ]:
# map plot v2

query2 = """
SELECT area_id, final_vuln_score, geometry
FROM assignment.vulnerability_final_scores
    JOIN assignment.nsw_shp ON (area_id = sa2_main16);
"""

gdf = gpd.read_postgis(query2, conn_wk09, geom_col='geometry')
sa2 = gpd.read_file('1270055001_sa2_2016_aust_shape/SA2_2016_AUST.shp')

ax = sa2.plot(color='white', edgecolor='grey', figsize=(20,20))

ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

bounds = gdf.geometry.bounds

plt.xlim([bounds.minx.min()-0.15, bounds.maxx.max()+0.15])
plt.ylim([bounds.miny.min()-0.15, bounds.maxy.max()+0.15])

gdf.plot(ax=ax, column='final_vuln_score', cmap='OrRd', legend=True,
                 legend_kwds={'label': "Vulnerability Score"})

plt.savefig('figure_vv_score.png')

In [ ]:
# map plot for positive cases

query2 = """
WITH
neighbourhoods_with_geom AS (
    SELECT area_id, gcc_code16, population, geometry
         FROM (assignment.neighbourhoods JOIN assignment.nsw_shp ON (area_id = sa2_main16))
),
postcode_with_geom AS (
SELECT postcode, AVG(longitude) AS avg_longitude, AVG(latitude) AS avg_latitude,
                ST_SetSRID(ST_MakePoint(AVG(longitude), AVG(latitude)), 4283) AS point
            FROM assignment.nsw_postcodes
            GROUP BY postcode
            ORDER BY postcode
),
cases_by_postcode_with_geom AS (
    SELECT postcode, positive_result_count, avg_longitude, avg_latitude, point
    FROM (SELECT postcode, result,
                COUNT(result) AS positive_result_count
            FROM assignment.covid_19_tests
            WHERE result = 'Case - Confirmed'
            GROUP BY postcode, result
            ORDER BY postcode) AS positive_cases
        JOIN postcode_with_geom USING (postcode)
)

SELECT area_id, SUM(positive_result_count) AS positive_result_count, geometry
FROM neighbourhoods_with_geom A LEFT JOIN cases_by_postcode_with_geom B
                    ON (ST_Intersects(A.geometry, B.point))
GROUP BY area_id, geometry
ORDER BY area_id;
"""

gdf = gpd.read_postgis(query2, conn_wk09, geom_col='geometry')
sa2 = gpd.read_file('1270055001_sa2_2016_aust_shape/SA2_2016_AUST.shp')

ax = sa2.plot(color='white', edgecolor='grey', figsize=(20,20))

ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

bounds = gdf.geometry.bounds

plt.xlim([bounds.minx.min()-0.15, bounds.maxx.max()+0.15])
plt.ylim([bounds.miny.min()-0.15, bounds.maxy.max()+0.15])

gdf.plot(ax=ax, column='positive_result_count', cmap='OrRd', legend=True,
                 legend_kwds={'label': "Number of positive cases"})

plt.savefig('figure_positive_cases.png')

In [ ]:
conn_wk09.close();